# Machine Learning in Python manual

<a name="contents"></a>
# Contents

- [Configuration](#conf)
- [Functions](#fnc)
- [Data Import](#imp)
- [Data Exploration](#expl)
- [Feature Engineering](#eng)
    - [Missing Values](#miss)
    - [Outliers](#out)
    - [Feature Encoding](#enc)
    - [Feature Scaling](#scal)
    - [Feature Selection](#sel)
    - [Dimensionality Reduction](#red)
    - [Preprocessing Pipelines](#pipe)
- [Machine Learning models](#ml)
    - [Models](#mlmodels)
    - [Prediction intervals](#mlint)
- [Deep Learning models](#dl)
- [Hyperparameters Tuning](#hyper)
    - [Cross Validation](#cv)
    - [Optuna](#optuna)
- [Performances](#perf)
    - [SHAP](#shap)
    - [Lift curve](#lift)
- [TensorFlow](#tf)

---
<a name="conf"></a>
# Configuration

[Return to Contents](#contents)

In [ ]:
import io
import os
import re
import math
import time
import pickle
import logging
import random as rnd
from typing import Dict, List
from datetime import timedelta
from operator import attrgetter

import joblib
import unidecode
import matplotlib
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import haversine as hs
from scipy import stats
from matplotlib import pyplot as plt
from IPython.display import display, HTML

from sklearn.datasets import load_iris, load_diabetes, load_digits, load_breast_cancer, fetch_california_housing, fetch_olivetti_faces
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, IsolationForest, RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, precision_recall_curve, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV  # , HalvingGridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier, XGBRegressor, plot_importance
# from catboost import CatBoostClassifier, CatBoostRegressor

# import tensorflow as tfb
# from tensorflow.keras import backend as K
# from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam

# from keras.callbacks import EarlyStopping
# from keras.wrappers.scikit_learn import KerasRegressor

from functions import plot_histrogram, plot_box, plot_violin, plot_distribution, check_outliers

### Notebook configuration

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
%matplotlib inline

### Logging & alerts configuration

In [ ]:
logging.basicConfig(
    # filename=logging_file,
    level=logging.DEBUG,  # logging.INFO
    format='%(asctime)s %(levelname)-8s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    # filemode='w'
)

logging.info('Starting notebook')

In [ ]:
# disable logging for matplotlib font
logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
%%javascript
alert('Configuration Complete')

Data generation and import functions

In [ ]:
def generate_data():
    """
    Generates data sample as seen in "Prediction Intervals for Gradient Boosting Regression"
    (https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html)
    """
    np.random.seed(1)
    f = lambda u: u * np.sin(u)

    #  first the noiseless case
    X_train = np.atleast_2d(np.random.uniform(0, 10.0, size=100)).T
    X_train = X_train.astype(np.float32)

    # observations
    y_train = f(X_train).ravel()
    dy = 1.5 + 1.0 * np.random.random(y_train.shape)
    noise = np.random.normal(0, dy)
    y_train += noise
    y_train = y_train.astype(np.float32)

    # mesh the input space for evaluations of the real function, the prediction and
    # its MSE
    X_test = np.atleast_2d(np.linspace(0, 10.0, 1000)).T
    X_test = X_test.astype(np.float32)
    y_test = f(X_test).ravel()


    return X_train, y_train, X_test, y_test

In [ ]:
# ---------------------------------------------------------------------
# # SINGLE PLOT
# # fpr, tpr, auc_thresholds = roc_curve(y, y_proba)
# # plot_roc_curve(fpr, tpr)
# plot_roc_curve(y, y_proba)

# # p, r, thresholds = precision_recall_curve(y, y_proba)
# # plot_precision_recall_vs_threshold(p, r, thresholds)
# plot_precision_recall_vs_threshold(y, y_proba)


# ---------------------------------------------------------------------
# # MULTIPLE PLOTS FOR THE SAME MODEL
# fig, axs = plt.subplots(ncols=2, figsize=(20, 8))

# # fpr, tpr, auc_thresholds = roc_curve(y, y_proba)
# # plot_roc_curve(fpr, tpr, ax=axs[0])
# plot_roc_curve(y, y_proba, ax=axs[0])

# # p, r, thresholds = precision_recall_curve(y, y_proba)
# # plot_precision_recall_vs_threshold(p, r, thresholds, ax=axs[1])
# plot_precision_recall_vs_threshold(y, y_proba, ax=axs[1])


# ---------------------------------------------------------------------
# MULTIPLE PLOTS FOR DIFFERENT MODELS
# fig, axs = plt.subplots(ncols=2, figsize=(20, 8))

# # first model
# fpr, tpr, auc_thresholds = roc_curve(y, y_proba)
# plot_roc_curve(fpr, tpr, label='First model', ax=axs[0])
# # second model
# fpr, tpr, auc_thresholds = roc_curve(y, y_proba_bis)
# plot_roc_curve(fpr, tpr, label='Second model', ax=axs[0])

# p, r, thresholds = precision_recall_curve(y, y_proba)
# plot_precision_recall_vs_threshold(p, r, thresholds, ax=axs[1])

---
<a name="imp"></a>
# Data Import

[Return to Contents](#contents)

In [ ]:
root_path = re.sub('(?<=python_examples)(.+)', '', os.getcwd())
python_path = os.path.join(root_path, 'python')
data_path = os.path.join(python_path, 'data')

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'housing.csv'))
# you can specify separator, columns types, null values
df = pd.read_csv(os.path.join(data_path, 'housing.csv'), sep=',', dtype={'ocean_proximity':str}, na_values=['null','Null','NULL','nan','NaN','NAN'], keep_default_na=False)

print(f'{len(df)} rows, {len(df.columns)} features')
df.head(1)

#### pubicly available datasets

Scikit learn provides a number of publicly available datasets which can be imported using sklearn methods

**References**
- https://scikit-learn.org/stable/datasets/toy_dataset.html
- https://inria.github.io/scikit-learn-mooc/python_scripts/trees_dataset.html

In [ ]:
# iris
dataset = load_iris()
print(f'{dataset.data.shape[0]} rows, {dataset.data.shape[1]} features')
display(pd.DataFrame(dataset.data, columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']).head(1))

# olivetti faces
dataset = fetch_olivetti_faces()
print(f'{dataset.images.shape[0]} images of shape {dataset.images.shape[1]}x{dataset.images.shape[2]} belonging to {len(np.unique(dataset.target))} people')

# california housing
dataset = fetch_california_housing(as_frame=True)
print(f'{len(dataset.frame)} rows, {len(dataset.frame.columns)} features')
display(dataset.frame.head(1))

---
<a name="expl"></a>
# Data Exploration

[Return to Contents](#contents)

In [ ]:
df.describe()
df.describe(include=['object'])  # ['int32','flat64','object','category']

In [ ]:
df.dtypes

### Distributions

In [ ]:
# plot_histrogram(df)
# plot_box(df)
# plot_violin(df)
plot_distribution(df, cols=['longitude','total_rooms'], plots=['hist','box'])

In [ ]:
# outliers
check_outliers(df)

### Skewness

In [ ]:
feature = 'median_house_value'

print(f'{feature} - skewness: {round(df[feature].skew(), 4)}')

plt.figure(figsize=(20,5))
sns.histplot(df[feature], kde=True, bins=100)
plt.title(f'{feature} - skewness: {round(df[feature].skew(), 4)}');

### Correlation

Correlation can be measured with correlation or or heatmap matrixes.  
Correlation can be computed directly in pandas with the `corr()` method and then can be easily inspected thanks to searborn `heatmap()` function.

**Note** that correlation only measures linear correlation hence it may completely miss out on nonlinear relationships

In [ ]:
corr_matrix = df.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).corr()
display(corr_matrix)

# to extract correlation with target feature
corr_matrix['median_house_value'].reset_index().sort_values(by='median_house_value', ascending=False)

In [ ]:
plt.figure(figsize=(20,5))
sns.heatmap(corr_matrix.abs(), annot=True);

### Scatter

In [ ]:
attributes = ['median_house_value','median_income','total_rooms','housing_median_age']
pd.plotting.scatter_matrix(df[attributes], figsize=(15, 8));

---
<a name="eng"></a>
# Feature Engineering

[Return to Contents](#contents)

In [ ]:
# map feature values using dictionary
df['total_bedrooms'] = df['total_bedrooms'].replace({'':np.nan})

# modify values based on rule
df.loc[df.total_bedrooms == '', 'total_bedrooms'] = np.nan

In [ ]:
# change feature type
df['total_bedrooms'] = df['total_bedrooms'].astype(float)

In [ ]:
# split dataset into train and target features
target_col = 'median_house_value'
X = df.drop(target_col, axis=1)
y = df[target_col]

print(X.shape)
X.head(1)

In [ ]:
# identify numeric and categorical features
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_cols = X.select_dtypes(include=num_types).columns
cat_cols = X.select_dtypes(['object']).columns.tolist()
print(f'Found {len(num_cols)} numeric columns and {len(cat_cols)} categorical columns')

In [ ]:
X.head(1)

<a name="miss"></a>
## Missing Values

[Return to Contents](#contents)

**Tips**:
- use kNN to impute missing values in numerical features (always, it is better than nn for continuous features)
- use deep learning imputation techniques when dealing with categorical features

In [ ]:
logging.info(f'{len([k for k,v in dict(X.isna().sum()).items() if v > 0])} features containing missing values')
X.isna().sum()

You should never consider dropping missing values as part of your data preparation step.  
Even knowing that some kind of information is missing for our data is a piece of information and you can use it to gain valuable insight on data

Anyway you can consider the following methods to drop missing data

In [ ]:
# to drop rows with missing values
X.dropna(subset=['total_bedrooms'])
# to drop the entire column
X.drop('total_bedrooms', axis=1).head(1)

Standard missing values imputation methods

In [ ]:
bedrooms_median = X['total_bedrooms'].median()
X['total_bedrooms'] = X['total_bedrooms'].fillna(bedrooms_median)
X.head(1)

Imputing missing valus with kNN can be done with the `KNNImputer`

**Note** that to train a `KNNImputer` train data must be numeric hence all categorical features have to be removed or encoded in order to be used

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputed_vals = knn_imputer.fit_transform(X[num_cols])
X = pd.concat([pd.DataFrame(imputed_vals, columns=num_cols), X[cat_cols]], axis=1)
assert X.isna().sum().max() == 0, 'ERROR! Found columns with missing values'
X.head(1)

<a name="out"></a>
# Outliers

[Return to Contents](#contents)

### z-Score

In statistics, the z-score it's a measure of how many standard deviations a raw score (meaning a data point, a sample) is above or below the population mean.  
It is calculated by subtracting the population mean from an individual raw score and then dividing the difference by the population standard deviation, hence, given a data point $x$ drawn from a population $N(μ,σ^{2})$ then the z-score for $x$ is computed as:
$$z = \frac{(x – μ)}{σ}$$

For example, a z-score of 2.5 indicates that the $x$ lies $2.5σ$ away from $μ$.

Remember that given a Gaussian distribution $N(μ,σ^{2})$:
- 68% of the data points lie between $\pmσ$ from $μ$
- 95% of the data points lie between $\pm2σ$ from $μ$
- 99.7% of the data points lie between $\pm3σ$ from $μ$


Usually $z=3$ is considered as a cut-off value to set a distinction between normal and anomalous data points.  
Therefore, any z-score greater than $+3$ or less than $-3$ is considered as outlier which is pretty much similar to standard deviation method.

In [ ]:
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_cols = X.select_dtypes(include=num_types).columns
cat_cols = X.select_dtypes(['object']).columns.tolist()
print(f'Found {len(num_cols)} numeric columns and {len(cat_cols)} categorical columns')

In [ ]:
# set the threshold for the z-score method
z_score_th = 3

# compute z-scores for all data points
z_score = np.abs(stats.zscore(X[num_cols]))

# filter data points based on their z-scores
X_outliers = X[(z_score[num_cols] >= z_score_th).any(axis=1)]
X_non_outliers = X[(z_score[num_cols] < z_score_th).all(axis=1)]

# remove target values corresponding to outliers
y_valid = y.loc[X_non_outliers.index]

assert all(X_non_outliers.index == y_valid.index), 'ERROR! Mismatching features and target'
print(f'Removed {len(X_outliers)} outlier observations, considering {len(X_non_outliers)} valid observations')
X_non_outliers.head(1)

### Isolation Forest

Isolation Forest is an unsupervised technique for identifying outliers using binary trees to detect anomalies, resulting in a linear time complexity and low memory usage that is well-suited for processing large datasets.  
The idea behind isolation forest is to identify as outliers those data points that are easily separable from the remaining data by linear splits, careless of whether those points are contained in data distribution or are outside.

It works by training binary trees on random samples drawn from the population. At each step, a random feature and a random threshold are selected to split data and these steps are repeated till each data point is completely isolated or until the max depth (if defined) is reached.  
Then, for each data point an anomaly score is computed based on the depth of the tree required to arrive at that data point.

In [ ]:
# define isolation forest model
if_model = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.001), random_state=123)
if_model.fit(X[num_cols])

# compute anomaly scores
X['anomaly_score'] = if_model.predict(X[num_cols])

# filter data points based on their anomaly scores
X_outliers = X[X['anomaly_score'] == -1]
X_non_outliers = X[X['anomaly_score'] == 1]

# remove target values corresponding to outliers
y_valid = y.loc[X_non_outliers.index]

assert all(X_non_outliers.index == y_valid.index), 'ERROR! Mismatching features and target'
print(f'Removed {len(X_outliers)} outlier observations, considering {len(X_non_outliers)} valid observations')
X_non_outliers.head(1)

<a name="enc"></a>
## Feature Encoding

[Return to Contents](#contents)

Although some machine learning models are able to deal with categorical values, most of the machine learning models can only work with numerical values.  
Feature encoding refers to the process of transforming categorical feature values into number that are more easily processed by a machine learning algorithm.

Note that there are 2 different types of categorical data which should be treated differently:
- ordinal data: data that comprises a finite set of discrete values with an order (es low, medium, high)
- nominal data: data that comprises a finite set of discrete values with no relationship between them (es india, congo, china)

**NOTE** apart from ordinal and nominal data there is actually also **cyclic** categorical data. Cyclic data refers to data that have a cyclic nature such as days of the week, months names...  
A common method for encoding cyclical data is to transform it into 2 dimensions using a sine and cosine transformation.

Here is a list of techiques that can be used for feature encoding:
- **Label Encoding**: it consists of substituting each group with a corresponding number and keeping such numbering consistent throughout the feature.  
This technique should be used when the categorical feature is ordinal since converting data to numbers creates relationship between them that do not exist in nominal category data.
- **Target Encoding**: it consists of substituting each group in a categorical feature with a numeric value based on the relationship between the category and the target variable (for example the mean).  
Target Encoding is a powerful solution also because it avoids generating a high number of features, as is the case for One-Hot Encoding, keeping the dimensionality of the dataset as the original one, making it suitable for models like decision trees and gradient boosting.
- **Leave-One-Out Encoding**: it encodes each category in a categorical variable with a numeric value based on the relationship between the category and the target variable excluding the current data point, which belongs to that category.  
This approach is similar to Target Encoding and it addresses the fact that Target Encoding can lead to target leakage or overfitting.
- **One-Hot Encoding**: it's consists of creating an additional feature for each group of the categorical feature and mark each observation belonging (value=1) or not (value=0) to that group.  
This approach is able to encode categorical features properly, despite some minor drawbacks. Specifically, the presence of a high number of binary values is not ideal for distance-based algorithms, such as Clustering models.  
In addition, the high number of additionally generated features can lead to the matrix being highly sparse (curse of dimensionality).
- **Hash Encoding**: it is a technique to represent categories similarly to One-Hot Encoding as a sparse matrix but with a much lower dimensionality.  
In feature hashing, a hashing function is applied to the category and then categories are represented using its indices
- **Binary Encoding**: it is a combination of Hash and One-Hot Encoding.  
It works really well where there is a high number of categories.


**Libraries**
- https://contrib.scikit-learn.org/category_encoders/index.html

**References**
- https://towardsdatascience.com/handling-categorical-data-the-right-way-9d1279956fc6
- https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159
- https://towardsdatascience.com/dealing-with-features-that-have-high-cardinality-1c9212d7ff1b

In [ ]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
import category_encoders as ce

In [ ]:
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num_cols = X.select_dtypes(include=num_types).columns
cat_cols = X.select_dtypes(['object']).columns.tolist()
print(f'Found {len(num_cols)} numeric columns and {len(cat_cols)} categorical columns')

### Label Encoding

In scikit-learn label encoding of feature variables can be done using the OrdinalEncoder class. scikit-learn also offers the LabelEncoder class that is specifically build to take care of the encoding of target variable and should not be used to encoder feature variables. 

In [ ]:
ordinal_encoder = OrdinalEncoder()
encoded_vals = ordinal_encoder.fit_transform(X[cat_cols])

X_encoded = pd.concat([X.drop(cat_cols, axis=1), pd.DataFrame(encoded_vals, columns=cat_cols)], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(1)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

### Target Encoding

In [ ]:
target_encoder = ce.TargetEncoder()
encoded_vals = target_encoder.fit_transform(X[cat_cols], y)

X_encoded = pd.concat([X.drop(cat_cols, axis=1), pd.DataFrame(encoded_vals, columns=cat_cols)], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(1)

### Leave-One-Out Encoding

In [ ]:
loo_encoder = ce.leave_one_out.LeaveOneOutEncoder(cols=cat_cols)
encoded_vals = loo_encoder.fit_transform(X, y)

X_encoded = pd.concat([X.drop(cat_cols, axis=1), pd.DataFrame(encoded_vals, columns=cat_cols)], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(1)

### One-Hot Encoding

In digital circuits and machine learning, a one-hot is a group of bits among which the legal combinations of values are only those with a single high bit (a 1) and all the others are low (0).

One-Hot Encoding therefore refers to the encoding method by which a new feature is created for every distinct value in the categorical feature. It can be implemented in pandas with the `get_dummies()` or the `OneHotEncoder()` functions. The difference between the 2 is that OneHotEncoder saves the exploded categories into it’s object. This is extremely important in the case that the unique values for the category feature are different between train and test set.

In [ ]:
# one-hot encoding with pandas get_dummies
encoded_vals = pd.get_dummies(X[cat_cols], drop_first=True)

# encoded_df = df.drop(cat_cols, axis=1)
X_encoded = pd.concat([X.drop(cat_cols, axis=1), encoded_vals], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(2)

In [ ]:
# one-hot encoding with sklearn OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_vals = one_hot_encoder.fit_transform(X[cat_cols])

X_encoded = pd.concat([X.drop(cat_cols, axis=1), pd.DataFrame(encoded_vals)], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(2)

### Hash Encoding

In [ ]:
# using scikit-learn
n_features = 5
hash_encoder = FeatureHasher(n_features=n_features, input_type='string')
encoded_vals = hash_encoder.transform(X[cat_cols].values)

X_encoded = pd.concat([X.drop(cat_cols, axis=1), pd.DataFrame(encoded_vals.toarray())], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(2)

In [ ]:
# using category_encoders library
n_components = 5
hash_encoder = ce.hashing.HashingEncoder(n_components=n_components)
encoded_vals = hash_encoder.fit_transform(X[cat_cols])

X_encoded = pd.concat([X.drop(cat_cols, axis=1), encoded_vals], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(2)

### Binary Encoding

In [ ]:
binary_encoder = ce.BinaryEncoder(cols=cat_cols)
encoded_vals = binary_encoder.fit_transform(X[cat_cols])

X_encoded = pd.concat([X.drop(cat_cols, axis=1), encoded_vals], axis=1)
print(f'Encoded dataframe results in {len(X_encoded)} rows and {len(X_encoded.columns)} columns')
X_encoded.head(2)

<a name="scal"></a>
## Feature Scaling

[Return to Contents](#contents)

Feature scaling refers to a family of techniques used in machine learning and data preprocessing that aim to bring different features of a dataset onto a similar scale.  
The goal is to ensure that all features contribute equally to the learning process and prevent any particular feature from dominating or biasing the results.

Among feature scaling techniques there are:
- **Normalization**: it maps input data to the interval $[0,1]$ (or $[-1,1]$ if negative values are present) while keeping the original data distribution. This method is highly impacted by outliers.
- **Standardization**: this technique assumes data is normally distributed and scales it so that the mean is 0 and the variance is 1. It is also referred to as **z-score** method. Note that this method is sensitive to outliers and to data skewness.
- **Robust standardization**: similar to standardization technique but uses the mediand and inter quantile range (IQR) instead of mean and variance being therefore robust against outliers.
- **Box-Cox transformation**: it belongs to a family of transformations called **power tranformations** and it aims to create a monotonic transformation of data using power functions to stabilize variance and make data more normally distributed.  
- **Skewness transformations**: they refer to a wide range of functions and methods used to transform data in order to remove skewness. You should select the best method depending on the particular data distribution and skeweness type. 

These techniques help in several ways:
- improved convergence: scaled data can help algorithms converge faster during training by preventing certain features from dominating the learning process due to their larger scales
- equalized impact: they ensure that all features contribute equally to the learning process, regardless of their original scales
- avoidance of numerical instability: they prevent numerical issues that can occur when features have extremely large or small values
- interpret ability: they allow for easier interpretation and comparison of feature importance

### Normalization

Normalization is a rescaling technique of data that maps original values to the range $[0, 1]$.  
This transformation is simply obtained using sample min and max values as follows

$$x_{scal}=\frac{x-x_{min}}{x_{max}-x_{min}}$$

Normalization works well when standard deviation is small but it is highly impacted by outliers.

In [ ]:
print(f'Performing normalization of {len(num_cols)} numerical features...')
minmax_scaler = MinMaxScaler()
scaled_feat = minmax_scaler.fit_transform(X[num_cols])
X_scaled = pd.DataFrame(scaled_feat, columns=num_cols, index=X.index)
# X[num_cols] = scaled_feat

# to revert scaling
minmax_scaler.inverse_transform(scaled_feat)

### Standardization

Standardization, also known as Z-score normalization or Z-score scaling, is a technique that involves rescaling the distribution of values so that the mean of observed values is 0 and the standard deviation is 1.  
This is obtained with the following transformation

$$x_{scal}=\frac{x-\bar{x}}{s}$$

where $\bar{x}$ is the sample mean and $s$ the sample standard deviation.

In [ ]:
print(f'Performing standard scaling of {len(num_cols)} numerical features...')
standard_scaler = StandardScaler()
scaled_feat = standard_scaler.fit_transform(X[num_cols])
X_scaled = pd.DataFrame(scaled_feat, columns=num_cols, index=X.index)
# X[num_cols] = scaled_feat

# to revert scaling
standard_scaler.inverse_transform(scaled_feat)

### Robust standardization

Standardization is a popular scaling technique that transforms the probability distribution for an input variable to a standard Gaussian (zero mean and unit variance). However it's important to note that standardization can produce skewed or biased data if the input variable contains outlier values.  
To overcome this, the median and interquartile range (IQR) can be used when standardizing numerical input variables. This method is generally referred to as robust standardization or robust scaling. In formula
$$x_{scal}=\frac{x-x_{median}}{IQR}$$

In [ ]:
print(f'Performing robust scaling of {len(num_cols)} numerical features...')
robust_scaler = RobustScaler()
scaled_feat = robust_scaler.fit_transform(X[num_cols])
X_scaled = pd.DataFrame(scaled_feat, columns=num_cols, index=X.index)
# X[num_cols] = scaled_feat

# to revert scaling
robust_scaler.inverse_transform(scaled_feat)

### Box-Cox transformation

The original Box-Cox tranformation is a one-dimensional transformation defined as

$$y_{i}^{\lambda}=\begin{cases}\frac{y_{i}^{\lambda} - 1}{\lambda} & \lambda != 0, \\ \log(y_{i}) & \lambda = 0 \end{cases}$$

where the optimal $\lambda$ is estimated via Maximum Likelihood Estimation (MLE).

The Box-Cox transformation is widely used for transforming features in order to make them more normally distributed. Note that when the original data distribution is far from being normal then this method might not work as you wish to.

In [ ]:
# scipy stats implementation of boxcox method estimates the optimal lambda but does not work with negative x
scaled_feat, lmbda = sp.stats.boxcox(X[num_cols])

# if you have data between -1 and 0 then you have to use scipy special implementation which however does not provide a method to estimate lambda
lam = .15
scaled_feat = sp.special.boxcox1p(X[num_cols], lam)

# to revert scaling
sp.special.inv_boxcox1p(scaled_feat, lam)

### Skeweness transformations

Skewness is a measure of asymmetry in a distribution of data.
Skewed data can be of 2 types
- right-skewed: also referred to as positively-skewed data, it's when data is clustered at low values
- left-skewed: also referred to as negatively-skewed data, it's when data is clustered at high values

There are 2 main methods to identify skewness in the data:
- observation method: this method relies on plotting the histogram of data and and look for statistical indicators like mean, median and mode.
For right-skewed data it holds that
$$mean > median > mode$$
while for left-skewed data it holds that
$$mean < median < mode$$
- statistical method: with this method we compute skewness as
$$skewness=\frac{3(mean - median)}{stddev}$$

Most of the statistical models do not work when the data is skewed and this is because the tail region of the skewed data distributions are outliers in the data and they can severely damage the performance of a statistical model.  
To deal with skewed data there are a bunch of techiques that are well represented by the ladder of powers that is
- cube power
- square power
- square root
- logarithm
- shifted logarithm
- reciprocal
- reciprocal square power

When data is right-skewed try moving down the ladder of powers to find the best transformation in order to remove skewness whereas if data is left-skewed try moving up the ladder of powers.

**Note** that this transformations require data to be positive. Some of them as log transformation in particular require data to be strictly positive

**References**
- http://seismo.berkeley.edu/~kirchner/eps_120/Toolkits/Toolkit_03.pdf

In [ ]:
# compute feature skewness
high_skew_th = .5
feat_skew = X[num_cols].apply(lambda x: sp.stats.skew(x.dropna())).reset_index().rename({'index': 'feature', 0: 'skew'}, axis=1)
high_skew_feat = feat_skew[(feat_skew['skew'] < -high_skew_th) | (feat_skew['skew'] > high_skew_th)]['feature'].tolist()

In [ ]:
# transformation ladder
c = high_skew_feat[0]

np.power(X[c], 3)
np.power(X[c], 2)
np.sqrt(X[c])                              # note that this transf has fixed point in 1
np.log(X[c])                               # note that this transf gives error if data is 0
np.log1p(X[c])                             # this one adds 1 to data in order to solve log issue
np.reciprocal(X[c])
np.power(X[c], -2)

Other transformation are **power transformations**. In statistics, power transform is a family of functions applied to create a monotonic transformation of data using power functions. It is a data transformation technique used to stabilize variance, make the data more normal distribution-like, improve the validity of measures of association (such as the Pearson correlation between variables), and for other data stabilization procedures.

A very useful case of power transform is **Box-Cox transformation** that follows from the inclusion of the $(λ − 1)$-th power of the geometric mean in the denominator. This simplifies the scientific interpretation of the equation because the units of measurement do not change as $λ$ changes. Box-Cox transformation is defined as follow:

$$\hat{y}_t = \begin{cases} log(y_{t}), & λ=0\\\frac{(y_{t}^{λ}-1)}{λ}, & λ!=0 \end{cases}$$

In [ ]:
# Box-Cox transf
# Box-Cox is available in scipy from scipy.stats.boxcox or scipy.special.boxcox
# the only difference between the 2 is that scipy.stats.boxcox estimates lambda if you do not provide it to the formula
# note that Box-Cox requires data to be strictly positive
bc_data, lamb = sp.stats.boxcox(X[c])     
bc_data = sp.special.boxcox(X[c])

# if you have null data then you can use either of the following 2 approaches
#  - use boxcox1p formula that applies the boxcox transformation to data plus 1
lam = .15
bc_data = sp.special.boxcox1p(X[c], lam)
#  - you can transform separately positive and null values. By definition Box-Cox transformation of a null value is 1/lambda where lambda is estimated by the model on positive values
pos_col = X[X[c] > 0][c]
bc_data, lamb = sp.stats.boxcox(pos_col)
bc_col = np.empty_like(X[c])
bc_col[X[c] > 0] = bc_data
bc_col[X[c] == 0] = -1 / lamb
X[c] = bc_col

# finally you can estimate the optimal Box-Cox transform parameter for input data with the following method
sp.stats.boxcox_normmax(X[c], method='mle')       # default method is pearson that maximizes the pearson correlation but mle is actually the one used in sp.stats.boxcox
sp.stats.boxcox_normmax(X[c] + 1, method='mle')   # add 1 if you are using boxcox1p

# to get the Box-Cox inverse transform use the following
sp.special.inv_boxcox(X[c], lam)
sp.special.inv_boxcox1p(X[c], lam)

<a name="sel"></a>
## Feature Selection

[Return to Contents](#contents)

Feature Selection is the process of selecting a subset of relevant features from a larger set of features in a dataset. The goal is to identify the most informative and discriminative features that contribute most to the predictive power of the model.

Note that when doing feature reduction you are actually removing some information from your data and this might also affect the model performances.

Feature selection techniques
- **Filter methods**: they rank features based on statistical metrics or heuristic measures. They assess the relevance of each feature independently of the learning algorithm. Popular methods are:
    - **Correlation-based Feature Selection (CFS)**: evaluates the correlation between features and the target variable
    - **Information Gain (IG)**: measures the reduction in entropy or impurity after including a particular feature
    - **Min-Redundancy Max-Relevance (MRMR)**: it works iteratively, selecting one feature at a time. At each step, it calculates the feature with the maximum score among the remaining unselected features. Note that it only works for supervised learning tasks
- **Wrapper methods**: they evaluate subsets of features by training and testing a specific ML model. They assess the performance of the model with different feature subsets to determine the optimal set of features. Common methods are:
    - **Recursive Feature Elimination (RFE)**: starts with all features and recursively eliminates the least important ones
    - **Genetic Algorithms (GA)**: uses an evolutionary algorithm to search for an optimal feature subset
    - **Boruta**: it creates shadow features which are shuffled copies of input features and combines them with the original ones. Then it runs a random forest classifier on the combined dataset and compute the z-scores for the variables. Finally it computes the maximum z-score among shadow features (MZSA) and keeps features that have significantly higher importance than MZSA while discarding features with significantly lower importance than MZSA
- **Embedded methods**: they incorporate feature selection within the model training process itself. The model automatically selects the most relevant features while learning the patterns in the data. Some examples are:
    - **L1 regularization (Lasso)**: it penalizes weights in proportion to the sum of the absolute values of the weights. In models relying on sparse features, L1 regularization helps drive the weights of irrelevant features to exactly 0, which removes those features from the model

### Information Gain

Information gain refers to a family of techniques that calculate the reduction in entropy or surprise when transforming a given dataset in some way.

When applied to feature selection it is referred to as **mutual information** as it measures dependency between 2 variables.

In [ ]:
from sklearn.feature_selection import mutual_info_classif, SelectKBest

In [ ]:
mutual_info = pd.Series(mutual_info_classif(X, y), index=X.columns)
mutual_info.sort_values(ascending=False)

In [ ]:
nr_features = 20

st = time.time()
mutual_info = SelectKBest(mutual_info_classif, k=5)
mutual_info.fit(X, y)
logging.info(f'Information Gain feature selection runtime: {timedelta(seconds=time.time() - st)}')
mutual_info.get_support()

### Recursive Feature Elimination (RFE)

RFE looks for a subset of features by starting with all features in the training dataset and iteratively discarding features until the desired number of features remains. This is achieved by fitting a ML algorithm to data, then ranking features by importance, discarding the least important features and re-fitting the model.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

In [ ]:
nr_features = 20

st = time.time()
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=nr_features)
rfe.fit(X, y)
logging.info(f'RFE feature selection runtime: {timedelta(seconds=time.time() - st)}')

X, y = rfe.transform(X, y)

### Stepwise Regression

Stepwise regression is the method of fitting regression models in which the choice of predictive variables is carried out by an automatic procedure. In each step, a variable is considered for addition to or subtraction from the set of explanatory variables based on some criterion.  
The primary advantage of stepwise regression is that it's **computationally efficient**. On the other side, its performances are generally worse than other methods. This is because, by making a hard selection on the next regressor and freezing the weight, it makes choices that are **locally optimal** at each step, but are **globally sub-optimal**.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector

In [ ]:
nr_features = 20

st = time.time()
sfs = SequentialFeatureSelector(LogisticRegression(), k_features=nr_features, forward=True, scoring='accuracy', cv=None)
selected_features = sfs.fit(X, y)
logging.info(f'Stepwise regression feature selection runtime: {timedelta(seconds=time.time() - st)}')

### Lasso

Lasso it's similar to stepwise regression but it has proven to yield better results. It penalizes the $L1$ norm of the weights, which induces sparsity in the solution. The degree of sparsity is controlled by the **penality term** that can be optimized through cross-validation or other optimization procedures.  
See [Lasso](#lasso) for details.

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# lasso feature selection
lasso_model = make_pipeline(RobustScaler(), Lasso())

search = GridSearchCV(
    lasso_model,
    {'model__alpha': np.arange(0.1,10,0.1)}, cv=5, scoring='neg_mean_squared_error', verbose=3
)

st = time.time()
search.fit(X_train, y_train)
logging.info(f'Lasso feature selection runtime: {timedelta(seconds=time.time() - st)}')
logging.info(f'Lasso penalty factor: {search.best_params_}')

# select features
coefficients = search.best_estimator_.named_steps['model'].coef_
importance = np.abs(coefficients)
selected_features = np.array(X_train.columns)[importance > 0]
discarded_features = np.array(X_train.columns)[importance == 0]

### Boruta

**Libraries**
- https://www.rdocumentation.org/packages/Boruta/versions/7.0.0/topics/Boruta

**References**
- https://towardsdatascience.com/simple-example-using-boruta-feature-selection-in-python-8b96925d5d7a

In [ ]:
from boruta import BorutaPy

In [ ]:
n_jobs = 4
forest = RandomForestRegressor(max_depth=5, n_jobs=n_jobs)
boruta = BorutaPy(estimator=forest, n_estimators='auto', verbose=1, max_iter=100, random_state=123)

print(f'Training Boruta on {len(non_iso_non_cat_cols)} features...')
st = time.time()
boruta.fit(np.array(X[non_iso_non_cat_cols]), np.array(y))
print(f'Boruta feature selection runtime: {timedelta(seconds=time.time() - st)}')

In [ ]:
# check selected features
boruta.support_

# check ranking of features
boruta.ranking_

In [ ]:
# divide features according to their color
green_area = list(compress(iso_walk_gross_cols, boruta_walk_gross.support_))
blue_area = list(compress(iso_walk_gross_cols, boruta_walk_gross.support_weak_))
red_area = [x for x in iso_walk_gross_cols if x not in green_area + blue_area]

### Min-Redundancy Max-Relevance (MRMR)

In [ ]:
from mrmr import mrmr_classif, mrmr_regression

In [ ]:
nr_features = 20

st = time.time()
# for classification tasks
selected_features = mrmr_classif(train_X[green_area+blue_area], train_y, K=nr_features, return_scores=True)
# for regression tasks
selected_features = mrmr_regression(train_X[green_area+blue_area], train_y, K=nr_features, return_scores=True)
logging.info(f'MRMR feature selection runtime: {timedelta(seconds=time.time() - st)}')

<a name="red"></a>
## Dimensionality Reduction

[Return to Contents](#contents)

Dimensionality reduction refers to techniques that transform a high-dimensional dataset into a lower-dimensional representation while preserving its essential structure and characteristics. The aim is to reduce the computational complexity, improve visualization, and eliminate redundant or noisy features.

Dimensionality reduction techniques
- **Principal Component Analysis (PCA)**: linear techinique that identifies a new set of orthogonal axes in the data, called principal components, which capture the maximum variance in the dataset. In PCA, the transformation is purely unsupervised, meaning that no information about the targets is used, therefore any highly predictive feature having low variance will be dropped
- **Partial Least Squares (PLS)**: supervised method similar to PCA. PLS will try to find the hyperplane in the dependant data that explains the maximum variance in the target data the main difference with PCA thus being that the identified hyperplane is the most descriptive of the target data. 
- **Linear Discriminant Analysis (LDA)**: supervised method commonly used in classification tasks. It aims to maximize the separability between different classes by finding a projection that maximizes the between-class scatter and minimizes the within-class scatter 
- **t-Distributed Stochastic Neighbor Embedding (t-SNE)**: non-linear technique particularly used for data visualization tasks. Its main characteristic is that it preserve the local structure of data while reducing the its dimension, meaning that data points that are close one to another in higher dimension space are represented close to each other in the low dimension space with high probability
- **Uniform Manifold Approximation and Projection (UMAP)**: non-linear algorithm that aims to represend data in a lower dimension space while preserving distance between data points

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
if perform_feat_red and feat_red_algo == 'PCA':
    print(f'Performing PCA feature reduction...')
    
    pca_df = X_feat_scal.copy()

    # identifying the best number of components for PCA
    pca = PCA(svd_solver='full')
    pca.fit(pca_df)

    # explained variance
    expl_var = pca.explained_variance_ratio_
    cum_expl_var = np.cumsum(expl_var)

    print(f'\t75% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .75])} features')
    print(f'\t80% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .80])} features')
    print(f'\t85% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .85])} features')
    print(f'\t90% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .90])} features')
    print(f'\t95% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .95])} features')
    print(f'\t99% of the variance can be attributed to {len([c for c in cum_expl_var if c <= .99])} features')

#     fig, ax = plt.subplots(figsize=(30,7))
#     ax.plot(cum_expl_var, label='expl variance')
#     ax.plot([.75]*len(expl_var), label='75%')
#     ax.plot([.90]*len(expl_var), label='90%')
#     ax.plot([.95]*len(expl_var), label='95%')
#     ax.plot([.99]*len(expl_var), label='99%')
#     ax.legend();
    
    num_feats = 102
    pca = PCA(n_components=num_feats, svd_solver='full')
    pca.fit(X_feat_scal)
    pca_samples = pca.transform(X_feat_scal)

    X_feat_red = pd.DataFrame(pca_samples)
    print(f'Reduced to {len(X_feat_red)} samples and {len(X_feat_red.columns.tolist())} features')

### PLS

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
n_dimensions = 5
pls = PLSRegression(n_components=n_dimensions)
pls.fit(X[num_cols], y)
X_red = pls.transform(X[num_cols])
print(f'Reduced X dimension: {X_red.shape}')

### LDA

Note that LDA makes the following assumptions on data
- each feature in the dataset follows a gaussian distribution
- each feature has the same variance
- lack of multicollinearity in independent features

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X, y)
transformed = lda.transform(X)

### t-SNE

**t-distributed Stochastic Neighbor Embedding (t-SNE)** is a statistical method for visualizing high-dimensional data by giving each datapoint a location in a two or three-dimensional map.  
It is a nonlinear dimensionality reduction technique well-suited for embedding high-dimensional data for visualization in a low-dimensional space of two or three dimensions. Specifically, it models each high-dimensional object by a two- or three-dimensional point in such a way that similar objects are modeled by nearby points and dissimilar objects are modeled by distant points with high probability.

A data point is a point $x_i$ in the original data space $\mathbf{R}^D$, where $D=64$ is the dimensionality of the data space. Every point is an image of a handwritten digit here. There are $N=1797$ points.

A map point is a point $y_i$ in the map space $\mathbf{R}^2$. This space will contain our final representation of the dataset. There is a bijection between the data points and the map points: every map point represents one of the original images.

How do we choose the positions of the map points? We want to conserve the structure of the data. More specifically, if two data points are close together, we want the two corresponding map points to be close too. Let's $\left| x_i - x_j \right|$ be the Euclidean distance between two data points, and $\left| y_i - y_j \right|$ the distance between the map points. We first define a conditional similarity between the two data points:

$$p_{j|i} = \frac{\exp\left(-\left| x_i - x_j\right|^2 \big/ 2\sigma_i^2\right)}{\displaystyle\sum_{k \neq i} \exp\left(-\left| x_i - x_k\right|^2 \big/ 2\sigma_i^2\right)}$$

This measures how close $x_j$ is from $x_i$, considering a Gaussian distribution around $x_i$ with a given variance $\sigma_i^2$. This variance is different for every point; it is chosen such that points in dense areas are given a smaller variance than points in sparse areas. The original paper details how this variance is computed exactly.

Now, we define the similarity as a symmetrized version of the conditional similarity:

$$p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N}$$

In [ ]:
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
import matplotlib.patheffects as PathEffects

In [ ]:
def tsne_scatter(x, y):
    # choose a color palette with seaborn
    palette = np.array(sns.color_palette("hls", 10))

    # create a scatter plot
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[y])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit
    txts = []
    for i in range(10):
        # position of each label.
        xtext, ytext = np.median(x[y == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
# import hand-written digits data
digits = load_digits()

# split images from labels
X = digits.data
y = digits.target

digits_proj = TSNE(random_state=1234).fit_transform(X)

tsne_scatter(digits_proj, y)

### UMAP

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(X)

def plot_UMAP():
    for i in range(10):
        plt.scatter(X_umap[y==i, 0], X_umap[y==i, 1], label=i, s=1)
    plt.xlabel('1st UMAP component')
    plt.ylabel('2nd UMAP component')
    plt.title('UMAP')
    plt.legend(loc=1)
    plt.grid()

plot_UMAP()

<a name="pipe"></a>
## Preprocessing Pipelines

[Return to Contents](#contents)

In [ ]:
pipeline = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder(), ['city']),
    ('label_encoder', LabelEncoder()),
    ('ordinal_encoder', OrdinalEncoder()),
    ('std_scaler', StandardScaler()),
    ('min_max_scaler', MinMaxScaler()),
    ('quant_transf', QuantileTransformer(n_quantiles=100, output_distribution='normal')),
    ('k_bins_discr', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')),
    ('null_replacer_knn', KNNImputer(n_neighbors=int(s))),
    ('vect', CountVectorizer(), 'title'),
    ('reduce_dim', PCA()),
    ('trunc_svd', TruncatedSVD(n_components=7)),
    ('random_forest_class', RandomForestClassifier())])
    
X_train_fitted = pipeline.fit_transform(X_train)

In order to train a Pipeline you first have to identify columns by data type.
This can be done with the following method

In [ ]:
numeric_cols = ...

---
<a name="ml"></a>
# Machine Learning models

[Return to Contents](#contents)

You should save every model you experiment with so that you can come back easily to any model you want.  
Make sure you save both the hyperparameters and the trained parameters, as well as the cross-validation scores and perhaps the actual predictions as well.
This will allow you to easily compare scores across model types, and compare the types of errors they make.
You can easily save Scikit-Learn models by using Python's `pickle` module or by using the `joblib` library, which is more efficient at serializing large NumPy arrays.

In [ ]:
X = df.drop('median_house_value', axis=1)
y = pd.DataFrame(df['median_house_value'])

In [ ]:
# splitting dataframes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=True, random_state=123)

# splitting indexes
# in this case you can later train the model and make predictions in the following manner
# model.fit(X.loc[ix_train,:], y.loc[iy_train,:])
ix_train, ix_test = train_test_split(X.index, stratify=y, shuffle=True, random_state=123)

<a name="mlmodels"></a>
## Models

[Return to Contents](#contents)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logr_model = LogisticRegression(
    tol=.00001,
    verbose=0,
    random_state=123)

st = time.time()
logr_model.fit(X_train, y_train)
logging.info(f'Logistic Regression model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = logr_model.predict(X_train)
y_val_pred = logr_model.predict(X_val)

In [ ]:
# save and load ml model
joblib.dump(lr_model, 'logistic_regression_model.pkl')
model = joblib.load('logistic_regression_model.pkl')

### Linear Regression

Linear regression fits a linear model with coefficients $\omega$ to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation.
Mathematically it solves a problem of the form

$$\min\limits_{\omega} \left( \left\|\omega X - y\right\|^2_2 \right)$$

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linr_model = LinearRegression()

st = time.time()
linr_model.fit(X_train, y_train)
logging.info(f'Linear Regression model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = linr_model.predict(X_train)
y_val_pred = linr_model.predict(X_val)

<a name="lasso"></a>
### Lasso

Lasso stands for **Least Absolute Shrinkage and Selection Operator** and it is a linear regression algorithm which performs $L1$ regularization to the coefficients meaning that it adds a penalty equal to the absolute value of the coefficients.  
The optimization objective for Lasso is:

$$\min\limits_{\omega} \left( \frac{1}{2n} \left\|\omega X - y\right\|^2_2 + \alpha\left\|\omega\right\|_1 \right)$$

that is

$$\min\limits_{\omega} \left(\sum_{i=1}^{n} \left(y_i - \sum_{j=1}^{p} x_{ij}\beta_{j}\right)^{2} + \alpha \sum_{j=1}^{p}\left|\beta_{j}\right| \right)$$

where $x$ is the train data, $y$ is the target data and $\beta$ are the linear regression coefficients.  
$n$ is the sample size and $p$ is the number of features for each train observation.

Technically the Lasso model is optimizing the same objective function as the Elastic Net without $L2$ penalty.

The Lasso procedure encourages simple, sparse models (ie models with fewer parameters). For this reason Lasso can also be used for variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model.  
Moreover Lasso is well-suited for models showing high levels of multicollinearity.

**Note** that Lasso is very sensitive to outliers so it is recommended to use a robust scaling algorithm prior to Lasso training for example sklearn RobustScaler.

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# regression
lasso_model = Lasso(alpha=0.0005)

st = time.time()
lasso_model.fit(X_train, y_train)
logging.info(f'Lasso model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = lasso_model.predict(X_train)
y_val_pred = lasso_model.predict(X_val)

In [ ]:
# robust scaling
# a simple method to automatically scaling features in a robust manner is using sklearn make_pipeline and RobustScaler
lasso_model = make_pipeline(RobustScaler(), Lasso(alpha=0.0005))

st = time.time()
lasso_model.fit(X_train, y_train)
logging.info(f'Lasso model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = lasso_model.predict(X_train)
y_val_pred = lasso_model.predict(X_val)

### Ridge

Ridge regression is a linear least squares algorithm with $L2$ regularization on the coefficients.  
The optimization objective for Ridge is:

$$\min\limits_{\omega} \left( \left\|\omega X - y\right\|^2_2 + \alpha\left\|\omega\right\|^2_2 \right)$$

that is

$$\min\limits_{\omega}\left(\sum_{i=1}^{n} \left(y_i - \sum_{j=1}^{p} x_{ij}\omega_{j}\right)^{2} + \alpha \sum_{j=1}^{p}\left(\omega_{j}\right)^{2}\right)$$

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# regression
ridge_model = Ridge(alpha=.001)

st = time.time()
ridge_model.fit(X_train, y_train)
logging.info(f'Ridge model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = ridge_model.predict(X_train)
y_val_pred = ridge_model.predict(X_val)

### Kernel Ridge

Kernel Ridge regression (**KRR**) combines Ridge regression with the kernel trick. It thus learns a linear function in the space induced by the respective kernel and the data. For non-linear kernels, this corresponds to a non-linear function in the original space.

In [ ]:
from sklearn.kernel_ridge import KernelRidge

In [ ]:
# regression
krr_model = KernelRidge(alpha=.001, kernel='polynomial', degree=2, coef0=2.5)

st = time.time()
krr_model.fit(X_train, y_train)
logging.info(f'Kernel Ridge model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = krr_model.predict(X_train)
y_val_pred = krr_model.predict(X_val)

### Bayesian Ridge

Bayesian regression techniques can be used to include regularization parameters in the estimation procedure in such a way that the regularization parameter is not set in a hard sense but tuned to the data at hand.

This can be done by introducing uninformative priors over the hyperparameters of the model. The $L2$ regularization used in Ridge regression and classification is equivalent to finding a maximum a posteriori estimation under a Gaussian prior over the coefficients $\omega$ with precision $\lambda^{-1}$.
Instead of setting $\lambda$ manually, it is possible to treat it as a random variable to be estimated from the data.

To obtain a fully probabilistic model, the output $y$ is assumed to be Gaussian distributed around $\omega X$

$$p\left(y|X,\omega,\alpha \right) = \mathcal{N}\left(y|\omega X,\alpha \right)$$

where $\alpha$ is again treated as a random variable that is to be estimated from the data.

The advantages of Bayesian Regression are:
- it adapts to the data at hand
- it can be used to include regularization parameters in the estimation procedure

whereas the disadvantages of Bayesian regression include:
- inference of the model can be time consuming.

Bayesian Ridge regression estimates a probabilistic model of the regression problem as described above.
The prior for the coefficient $\omega$ is given by a spherical Gaussian

$$p\left(\omega|\lambda \right) = \mathcal{N}\left(\omega|0,\lambda^{-1}I_p \right)$$

The priors over $\alpha$ and $\lambda$ are chosen to be gamma distributions, the conjugate prior for the precision of the Gaussian. The resulting model is called Bayesian Ridge Regression, and is similar to the classical Ridge.

In [ ]:
from sklearn.linear_model import BayesianRidge

In [ ]:
# regression
brr_model = BayesianRidge(alpha=.001)

st = time.time()
brr_model.fit(X_train, y_train)
logging.info(f'Bayesian Ridge model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = brr_model.predict(X_train)
y_val_pred = brr_model.predict(X_val)

### Automatic Relevance Determination

Automatic Relevance Determination (**ARD**) regression is very similar to Bayesian Ridge Regression, but can lead to sparser coefficients $\omega$.

It poses a different prior over $\omega$, by dropping the assumption of the Gaussian being spherical.
Instead, the distribution over $\omega$ is assumed to be an axis-parallel, elliptical Gaussian distribution.
This means each coefficient $\omega_i$ is drawn from a Gaussian distribution, centered on zero and with a precision $\lambda_i$

$$p\left(\omega|\lambda \right) = \mathcal{N}\left(\omega|0,A^{-1} \right)$$

with $diag(A) = \lambda = \{\lambda_1, ..., \lambda_p\}$.

In contrast to Bayesian Ridge regression, each coordinate of $\omega_i$ has its own standard deviation $\lambda_i$.

In [ ]:
from sklearn.linear_model import ARDRegression

In [ ]:
# regression
ard_model = ARDRegression(alpha=.001, n_iter=1000, tol=1e-4)

st = time.time()
ard_model.fit(X_train, y_train)
logging.info(f'ARD model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = ard_model.predict(X_train)
y_val_pred = ard_model.predict(X_val)

### ElasticNet

The ElasticNet is a regularized regression method that linearly combines the $L1$ and $L2$ penalties of the Lasso and Ridge methods.    
The optimization objective for ElasticNet is:

$$\min\limits_{\omega} \left( \frac{1}{2n}\left\|\omega X - y\right\|^2_2 + \alpha\rho\left\|\omega\right\|_1 + \frac{1}{2}\alpha(1-\rho)\left\|\omega\right\|^2_2 \right)$$

that is

$$\min\limits_{\omega} \left( \frac{1}{2n} \sum_{i=1}^{n} \left(y_i - \sum_{j=1}^{p} x_{ij}\omega_{j} \right)^{2} + \alpha \rho \sum_{j=1}^{p}\left|\omega_{j} \right| + \frac{1}{2} \alpha (1 - \rho) \sum_{j=1}^{p}\left(\omega_{j}\right)^{2} \right)$$

where $x$ is the train data, $y$ is the target data and $\beta$ are the linear regression coefficients.  
$n$ is the sample size and $p$ is the number of features for each train observation.  
$\rho$ is called the $l1$ ratio and it is the ElasticNet mixing parameter. It ranges between 0 and 1 included with the limit cases being Lasso when it is 1 and a pure $L2$ penalty when it is 0.

**Note** that similarly to Lasso, ElasticNet is very sensitive to outliers.

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
# regression
enet_model = ElasticNet(alpha=0.0005, l1_ratio=.9)

st = time.time()
enet_model.fit(X_train, y_train)
logging.info(f'Elastic Net model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = enet_model.predict(X_train)
y_val_pred = enet_model.predict(X_val)

### Naive Bayes

Naive Bayes refers to a family of classification algorithms based on applying Bayes' theorem with the "naive" assumption of conditional independence between every pair of features given the value of the class variable.

Bayes' theorem states that, given a class variable $y$ and dependent feature vector $X$:

$$P(y|X) = \frac{P(y)P(X|y)}{P(X)}$$

Using the naive conditional independence assumption, meaning that

$$P(x_i|y,x_1,...x_{n-1},x_{n+1},...,x_n) = P(x_i|y)$$

for all $i$, Bayes relationship is simplified to

$$P(y|X) = \frac{P(y)\prod_{i=1}^nP(x_i|y)}{P(X)}$$

Now, since $P(X)$ is constant given the input, we can derive the following

$$P(y|X)\propto P(y)\prod_{i=1}^nP(x_i|y)$$

from which it follows that

$$\hat y = arg\max\limits_y P(y)\prod_{i=1}^nP(x_i|y)$$

and we can use Maximum A Posteriori (MAP) estimation to estimate $P(y)$ and $P(x_i|y)$, the former being the relative frequency of class $y$ in the training set.

The different Naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$.

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# gaussian naive bayes
gnb_model = GaussianNB()

st = time.time()
gnb_model.fit(X_train, y_train)
logging.info(f'Gaussian Naive Bayes model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = gnb_model.predict(X_train)
y_val_pred = gnb_model.predict(X_val)

### Support Vectore Machine

A Support Vector Machine (**SVM**) ...

The objective of the SVM algorithm is to find the **maximimum-margin hyperplane** in a $p$-dimensional space where $p$ is the dimension of the data. Maximum-margin refers to the request that the hyperplane is defined so that the distance between the minimum data points in the 2 semiplanes is maximized.

In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using what is called the **kernel trick**, implicitly mapping their inputs into high-dimensional feature spaces.

In [ ]:
from sklearn.svm import SVC, SVR

In [ ]:
svm_params = {
    kernel='rbf',
    degree=3,
    verbose=0
}

# classification
svc_model = SVC(**svm_params)

# regression
svr_model = SVR(**svm_params)

st = time.time()
svr_model.fit(X_train, y_train)
logging.info(f'SVR model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = svr_model.predict(X_train)
y_val_pred = svr_model.predict(X_val)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [ ]:
tree_params = {
    max_depth:3,
    min_samples_split:3,
    random_state:123
}

# classification
tree_model = DecisionTreeClassifier(**tree_params)

# regression
tree_model = DecisionTreeRegressor(**tree_params)

st = time.time()
tree_model.fit(X_train, y_train)
logging.info(f'Decision Tree model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = tree_model.predict(X_train)
y_val_pred = tree_model.predict(X_val)

In [ ]:
# creating the tree plot
plt.figure(figsize=(40, 10))
plot_tree(tree_model, feature_names=X_train.columns.tolist(), filled=True, fontsize=12)
plt.rcParams['figure.figsize'] = [20, 10]
plt.savefig(f'tree_decision_split.jpg')

### Random Forest

Random Forest is an ensemble learning algorithm based on decision trees. The ensemble model is built with **bagging** technique by **boostrapping the data** by randomly choosing subsamples of the train data and training a decision tree on each. Decision trees are trained in parallel and the final prediction is made by averaging the predicitons made by all the decision trees. 

**Note** that Random Forest is based on trees whose leaves contain only constant values. This means that a given random forest model or any other tree based model can only predict a discrete number of values. This number of values is called **cardinality** of the model.  
Cardinality is fundamental for the following reasons:
- if the model cardinality is much lower that the cardinality of the prediction set then the model is unable to capture the underlying data trend correctly. T
he set of predictions is not large enough to capture the complexity of data;
- if the model cardinality is much higher than the cardinality of the prediction set then the model is likely overfitting.

**Note** that tree-based ensemble algorithms are **non-parametric models**. This means that they do NOT assume or require data to follow a specific distribution and therefore they do not features to be independent (allowing for multicollinearity in data), they do not require data to be scaled and they are usually robust to outliers and to overfitting.

**References**
- https://towardsdatascience.com/the-ultimate-guide-to-adaboost-random-forests-and-xgboost-7f9327061c4f

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
rf_params = {
    n_estimators:700,
    max_depth:7,
    verbose:0,
    random_state:123
}

# classification
rf_model = RandomForestClassifier(**rf_params)

# regression
rf_model = RandomForestRegressor(**rf_params)

st = time.time()
rf_model.fit(X_train, y_train)
logging.info(f'Random Forest model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = rf_model.predict(X_train)
y_val_pred = rf_model.predict(X_val)

To get the depth of trees in a random forest use one of the following

In [ ]:
list(set([estimator.tree_.max_depth for estimator in rf_model.estimators_]))
list(set([estimator.get_depth() for estimator in rf_model.estimators_]))

<a name="adaboost"></a>
### AdaBoost

AdaBoost stands for **Adaptive Boosting** and it is an ensemble learning algorithm based on decision trees.  
Differently from Random Forest, the ensemble model is built with boosting technique by sequentially growing decision trees as weak learners and punishing the incorrectly predicted samples by assigning a larger weight to them after each round of prediction. This way, the algorithm is learning from previous mistakes.  
The final prediction is made by averaging the trees prediction.

The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction. 

**References**
- https://medium.com/@divyagera2402/boosting-algorithms-adaboost-gradient-boosting-xgb-light-gbm-and-catboost-e7d2dbc4e4ca

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostClassifier

In [ ]:
# classification
ada_params = {
    'n_estimators': 400,
    'learning_rate' : 0.65
}
ada_model = AdaBoostClassifier(**ada_params)

st = time.time()
ada_model.fit(X_train, y_train)
logging.info(f'AdaBoost model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = ada_model.predict(X_train)
y_val_pred = ada_model.predict(X_val)

<a name="gboost"></a>
### Gradient Boosting

Gradient Boosting is similar to AdaBoost as it is a tree-based ensemble learning algorithm.  
The main difference with [AdaBoost](#adaboost) lies in what Gradient Boosting does with the underfitted values of its predecessor. Contrary to AdaBoost, which tweaks the instance weights at every interaction, this method tries to fit the new predictor to the residual errors made by the previous predictor.  

**Note** that in regression task the default loss function is `squared_error` but `quantile` can be used to estimate prediction intervals (see [Prediction intervals](#regint) for details).

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [ ]:
gb_params = {
    n_estimators:500,
    max_depth:5,
    learning_rate:0.01,
    min_samples_leaf:5,
    min_samples_split:5,
    random_state:12
}

# classification
gb_model = GradientBoostingClassifier(**bg_params)

# regression
gb_model = GradientBoostingRegressor(**bg_params)

st = time.time()
gb_model.fit(X_train, y_train)
logging.info(f'Gradient Boosting model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = gb_model.predict(X_train)
y_val_pred = gb_model.predict(X_val)

<a name="xgboost"></a>
### XGBoost

XGBoost stands for **eXtreme Gradient Boosting** and it is an advanced implementation of [Gradient Boosting](#gboost).  
Extreme refers to the fact that this implementation of Gradient Boosting algorithm is designed to "*push the extreme of the computation limits of machines to provide a scalable , portable and accurate library.*”

XGBoost grows **asymmetric trees** meaning that splitting condition for each node across the same depth can differ. Trees are grown **level-wise** (vertically) meaning that at each step all leaf nodes are exploded and evaluated. Splitting method ... XGBoost does not support categorical features, does not support text features and can handle missing values.

XGBoost is among the best algorithms when dealing with classification problems.  
On the other hand, XGBoost is not recommended when dealing with regression tasks that involve predicting a continuous variable, in tasks involving extrapolation or tasks where the target variable has values that are not present in the training data. This is because XGBoost is not able to predict values that it hasn't seen in training data.

**Note** that in classification, XGBoost performances scale quadratically with the number of classes.

**Libraries**
- https://xgboost.readthedocs.io/en/stable/

**References**
- https://towardsdatascience.com/catboost-vs-lightgbm-vs-xgboost-c80f40662924

In [ ]:
from xgboost import XGBClassifier, XGBRegressor, plot_importance

In [ ]:
xgb_params = {
    n_estimators:700,
    max_depth:7,
    learning_rate:0.01,
    gamma:0.1,
    verbosity:0,
    random_state:123
}

# regression
xgb_model = XGBRegressor(xgb_params)

st = time.time()
xgb_model.fit(X_train, y_train)
logging.info(f'XGBoost model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = xgb_model.predict(X_train)
y_val_pred = xgb_model.predict(X_val)

<a name="lgbm"></a>
### LightGBM

LightGBM stands for **Light Gradient Boosting Machine** and it's a [Gradient Boosting](#gboost) framework that uses tree based learning algorithms designed to be distributed and efficient allowing for faster parallel train, lower memory usage and better accuracy.

Similarly to [XGBoost](#xgboost), it grows **asymmetric trees** but here instead trees are grown **leaf-wise** (horizontally) resulting in smaller and faster models compared to XGBoost.


**Libraries**
- https://lightgbm.readthedocs.io/en/v3.3.2/

In [ ]:
lgb_params = {
    n_estimators:720,
    num_leaves:5,
    learning_rate:0.05,
    max_bin:55,
    bagging_fraction:0.8,
    bagging_freq:5,
    min_data_in_leaf:6,
    min_sum_hessian_in_leaf:11
}

# regression
lgb_model = lgb.LGBMRegressor(objective='regression', **lgb_params)

st = time.time()
lgb_model.fit(X_train, y_train)
logging.info(f'LightGBM model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = lgb_model.predict(X_train)
y_val_pred = lgb_model.predict(X_val)

In [ ]:
# categorical feature importance
for col in cat_cols:
    X[col] = pd.factorize(X[col])[0]
    X[col] = X[col].astype('category')
    
# lgbm
train_set = lgb.Dataset(X[cat_cols], label=y)
params = {'n_estimators':100, 'max_depth':5, 'learning_rate':0.1, 'num_leaves':500}
lgbm = lgb.train(params=params, train_set=train_set, categorical_feature=cat_cols)

print('Plot feature importances...')
plt.figure(figsize=(30,15))
ax = plt.subplot(111)
lgb.plot_importance(lgbm, ax=ax);

<a name="catboost"></a>
### CatBoost

CatBoost is a more recent implementation of [Gradient Boosting](#gboost).

It differs from [XGBoost](#xgboost) and [LightGBM](#lgbm) since it grows **symmetric trees** meaning that the splitting condition is consistent across all nodes at the same depth of the tree and therefore the splitting condition must result in the lowest loss across all nodes of the same depth. CatBoost supports numerical, categorical and text features and can handle missing values.

`Quantile` loss function can be used to estimate prediction intervals (see [Prediction intervals](#regint) for details)

**Note** that CatBoost train time can increase a lot with trees' depth.

**Libraries**
- https://catboost.ai/en/docs/

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor

In [ ]:
catb_params = {
    n_estimators:700,
    depth:7,
    learning_rate:0.1,
    l2_leaf_reg:0.5,
    verbose:0,
    random_state:123
}

# classification
catb_model = CatBoostClassifier(**catb_params)

# regression
catb_model = CatBoostRegressor(**catb_params)

st = time.time()
catb_model.fit(X_train, y_train)
logging.info(f'CatBoost model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = catb_model.predict(X_train)
y_val_pred = catb_model.predict(X_val)

Feature importance can be found thanks to the `get_feature_importance()` method.  
The output feature importance depends on the specified `type`, specifically:
- **PredictionValuesChange**: (default) for each feature it shows how much on average the prediction changes if the feature value changes, the bigger the value of the importance the bigger on average is the change to the prediction value, if this feature is changed;
- **LossFunctionChange**: for each feature the value represents the difference between the loss value of the model with this feature and without it. Since it is computationally expensive to retrain the model without one of the features, this model is built approximately using the original model with this feature removed from all the trees in the ensemble. This type is particularly effective for **ranking models**;
- **PredictionDiff**: for each feature it reflects the maximum possible change in the predictions difference if the value of the feature is changed for both objects. The change is considered only if there is an improvement in the direction of changing the order of documents.

In [ ]:
catboost = CatBoostRegressor(max_depth=5) 
catboost.fit(X, y, verbose=False)
catboost_feat_imp = catboost.get_feature_importance()

# create df to inspect
catboost_features = pd.DataFrame(X.columns, columns=['feature'])
catboost_features['importance'] = catboost_feat_imp
catboost_features = catboost_features.sort_values('importance', ascending=False).reset_index().reset_index().rename({'index':'org_order', 'level_0':'importance_order'}, axis=1)
catboost_features['cumulative_importance'] = np.cumsum(catboost_features['importance'])
display(catboost_features[['feature','importance','cumulative_importance']].head())

# take the features that account for 95% of importance
n_features = catboost_features[catboost_features.cumulative_importance < 95].iloc[-1].importance_order + 1
print(f'Selected {n_features} features')

plt.figure(figsize=(40,7))
ax = plt.subplot(111)
ax.plot(catboost_features.importance_order, catboost_features.cumulative_importance, label='cumulative importance')
ax.axhline(y=95, color='r', linestyle='-', label='95% importance')
ax.axvline(x=n_features, color='r', linestyle='-', label='selected features')
ax.legend();

Moreover, if you want to reduce the size of the train dataset to a given number of features you can also take advantage of the `select_features()` method

This method iteratively discards features from the train set in order to reach the input number of features.  
Finally it plots the loss by the eliminated features.

In [ ]:
catboost = CatBoostRegressor(max_depth=5) 
catboost_selected_feats = catboost.select_features(
    X, y, features_for_select=X.columns, num_features_to_select=100, verbose=False, plot=True
)

### Ensemble

Ensemble models is a machine learning approach to combine multiple other models in the prediction process. Those models are referred to as base estimators.

It is a solution to overcome the following technical challenges of building a single estimator:
- high variance: the model is very sensitive to the provided inputs to the learned features;
- low accuracy: one model or one algorithm to fit the entire training data might not be good enough to meet expectations;
- features noise and bias: the model relies heavily on one or a few features while making a prediction.

There are different ensemble techniques such as:
- **bagging**: multiple models are trained seprately on different subsets of the training data. This approach reduces variance and minimizes overfitting.
- **boosting**: in this technique multiple models are trained sequentially so that each new model is trained on the error produced by the previous model. This approach reduces bias.
- **stacking**:
- **blending**:

#### Stacking

The general framework of a stacked ensemble consists of two or more **base models** (level-0 models) and a higher level **meta model** (level-1 model) with their functions as below:
- base models: models that fit the training data and predict out-of-sample data;
- meta model: model that fits on the prediction from base-models and learns how to best combine the predictions.

Stacking is appropriate when multiple different machine learning models have skill on a dataset, but have skill in different ways. Another way to say this is that the predictions made by the models or the errors in predictions made by the models are uncorrelated or have a low correlation.

Base models are often complex and diverse. As such, it is often a good idea to use a range of models that make very different assumptions about how to solve the predictive modeling task, such as linear models, decision trees, support vector machines, neural networks, and more. Other ensemble algorithms may also be used as base-models, such as random forests.

There hasn't been any research on how to choose or optimize the meta model in this case. In most cases, the meta-model used is just a simple model such as Linear Regression for regression tasks and Logistic Regression for classification tasks. One reason why more complex meta-models are often not chosen is because there is a much higher chance that the meta-model may overfit to the predictions from the base models.

**References**
- https://www.kaggle.com/code/daisukelab/optimizing-ensemble-weights-using-simple
- https://towardsdatascience.com/stacked-ensembles-improving-model-performance-on-a-higher-level-99ffc4ea5523

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
# stacked ensemble
estimators = [
    ('ridge', RidgeCV()),
    ('lasso', LassoCV(random_state=42)),
    ('knr', KNeighborsRegressor(n_neighbors=20, metric='euclidean'))
]

stack_reg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

st = time.time()
stack_reg.fit(X_train, y_train)
logging.info(f'Stacked ensemble model train runtime: {timedelta(seconds=time.time() - st)}')

y_train_pred = stack_reg.predict(X_train)
y_val_pred = stack_reg.predict(X_val)

<a name="mlint"></a>
## Prediction intervals

[Return to Contents](#contents)

#### Random Forest

**Libraries**
- https://contrib.scikit-learn.org/forest-confidence-interval/

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=700,
    max_depth=7,
    verbose=0,
    random_state=123)

st = time.time()
rf_model.fit(X_train, y_train)

plt.figure(figsize=(35,10))
ax = plt.subplot(111)
ax.errorbar(y_val, rf_sup.predict(X_va), yerr=np.sqrt(rf_model), fmt='o')
ax.plot(y_val, y_val, 'r')
ax.xlabel('Actual')
ax.ylabel('Predicted')

#### Gradient Boosting

**References**
- https://medium.com/walmartglobaltech/adding-prediction-intervals-to-tree-based-models-8ea53814a4b9

In [ ]:
all_models = {}
all_predictions = {}
common_params = dict(
    learning_rate=0.01,
    n_estimators=500,
    max_depth=5,
    min_samples_leaf=5,
    min_samples_split=5,
    verbose=0,
    random_state=123
)

for alpha in [0.05, 0.10, 0.5, 0.90, 0.95]:
    gb_model = GradientBoostingRegressor(loss='quantile', alpha=alpha, **common_params)
    # all_models["q %1.2f" % alpha] = gb_model.fit(X_train, y_train)
    # all_models['q {:.2f}'.format(alpha)] = gb_model.fit(X_train, y_train)
    all_models['q_%02.f' % (alpha * 100)] = gb_model.fit(X_train, y_train)
    all_predictions['q_%02.f' % (alpha * 100)] = all_models['q_%02.f' % (alpha * 100)].predict(X_val)

gb_mse_model = GradientBoostingRegressor(loss='squared_error', **common_params)
all_models['mse'] = gb_mse_model.fit(X_train, y_train)
all_predictions['mse'] = all_models['mse'].predict(X_val)

In [ ]:
nr_points = 100
xx = np.linspace(1, nr_points, nr_points)

fig = plt.figure(figsize=(30, 7))
ax = plt.subplot(111)
# plt.plot(xx, f(xx), "g:", linewidth=3, label=r"$f(x) = x\,\sin(x)$")
ax.plot(xx, y_val[:nr_points], 'b.', label='Observation')
ax.plot(xx, all_predictions['mse'][:nr_points], 'r.', label='Predictions')
ax.plot(xx, all_predictions['q_90'][:nr_points], 'k-')
ax.plot(xx, all_predictions['q_10'][:nr_points], 'k-')
plt.fill(
    np.concatenate([xx, xx[::-1]]),
    np.concatenate([all_predictions['q_90'][:nr_points], all_predictions['q_10'][:nr_points][::-1]]),
    alpha=.2, fc='b', ec='None', label='80% prediction interval')
ax.legend();

#### CatBoost

In [ ]:
all_models = {}
all_predictions = {}
common_params = dict(
    n_estimators=700,
    depth=7,
    learning_rate=0.01,
    l2_leaf_reg=0.5,
    verbose=0,
    random_state=123
)

for alpha in [0.05, 0.10, 0.5, 0.90, 0.95]:
    catb_model = CatBoostRegressor(loss_function='Quantile:alpha={}'.format(alpha), **common_params)
    all_models['q_%02.f' % (alpha * 100)] = catb_model.fit(X_train, y_train)
    all_predictions['q_%02.f' % (alpha * 100)] = all_models['q_%02.f' % (alpha * 100)].predict(X_val)

catb_mse_model = CatBoostRegressor(**common_params)
all_models['mse'] = catb_mse_model.fit(X_train, y_train)
all_predictions['mse'] = all_models['mse'].predict(X_val)

In [ ]:
nr_points = 100
xx = np.linspace(1, nr_points, nr_points)

fig = plt.figure(figsize=(30, 7))
ax = plt.subplot(111)
# plt.plot(xx, f(xx), "g:", linewidth=3, label=r"$f(x) = x\,\sin(x)$")
ax.plot(xx, y_val[:nr_points], 'b.', label='Observation')
ax.plot(xx, all_predictions['mse'][:nr_points], 'r.', label='Predictions')
ax.plot(xx, all_predictions['q_90'][:nr_points], 'k-')
ax.plot(xx, all_predictions['q_10'][:nr_points], 'k-')
plt.fill(
    np.concatenate([xx, xx[::-1]]),
    np.concatenate([all_predictions['q_90'][:nr_points], all_predictions['q_10'][:nr_points][::-1]]),
    alpha=.2, fc='b', ec='None', label='80% prediction interval')
ax.legend();

---
<a name="dl"></a>
# Deep Learning models

[Return to Contents](#contents)

#### Feed-Forward Neural Networks

**Note** that there are several rules of thumb methods for determining the correct number of neurons in a hidden layer such as:
- it should be between the size of the input layer and the size of the output layer
- it should be 2/3 or the size of input layer plus the size of the output layer
- it should be less than twice the size of the input layer

**References**
- https://medium.com/geekculture/introduction-to-neural-network-2f8b8221fbd3
- https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594

In [ ]:
# classification
# clear tf session
K.clear_session()

# initialize nn
nn = Sequential()

# add layers
nn.add(Dense(128, activation='relu', input_dim=n_features))
nn.add(Dropout(.7))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(.7))
nn.add(Dense(30, activation='relu'))
nn.add(Dropout(.7))
nn.add(Dense(1, activation='sigmoid'))

nn.compile(loss='mean_squared_error', optimizer=Adam(lr=.002), metrics=['accuracy'])
perf = nn.fit(X_train, y_train, epochs=500, batch_size=64, validation_data=(X_val, y_val), verbose=0)

In [ ]:
# regression


---
<a name="hyper"></a>
# Hyperparameters Tuning

[Return to Contents](#contents)

<a name="cv"></a>
## Cross Validation

[Return to Contents](#contents)

Cross Validation is a resampling technique that can be used to evaluate and select machine learning algorithms on a limited dataset.
k-fold cross validation is a type of cross validation, where the training data is split into k-folds and (k-1) folds is used for training and k-th fold is used for validation of the model.

Common scoring metrics for **regression** are 
- neg_mean_absolute_error
- neg_mean_squared_error
- neg_root_mean_squared_error
- neg_mean_squared_log_error
- r2

Common scoring metrics for **classification** are 
- accuracy
- f1
- precision
- recall
- roc_auc

To inspect sklearn scoring metrics run `sklearn.metrics.SCORERS.keys()`

In [ ]:
cross_val_score(model, X, y, cv=3, scoring=)

### Grid Search Cross Validatoin

This method allows to try out all possible combinations of hyperparameters given a range of values for each of them.
It uses cross validation to evaluate the performances of the model for a given combination of hyperparameters and then return the hyperparameters that are scoring the best result.

You can run a Grid Search Cross Validation with the `GridSearchCV` method.
This method will compute a model for every hyperparameter combination and each model will be trained n times where n is specified by the `cv` parameter.
Hyperparameter ranges have to be provided to the method as a dictionary.
You can also provide a dictionary of dictionaries in order to tell the Grid Search method to inspect the different hyperparameter combinations separately.

**TIP**: when you have no idea what value a hyperparameter should have, a simple approach is to try out consecutive powers of 10

**NOTE** that you can treat some of the data preparation steps as hyperparameters and that `GridSearchCV` can also be used to evaluate whether or not to add a feature, for outliers handling, for missing values imputation and more.

In [ ]:
n_iter_search = 10
param_distr = {
    'n_estimators': [50, 100],
    'max_depth': [3,4],
}

grid_search = GridSearchCV(
    RandomForestRegressor(verbose=0, random_state=123),
    param_grid=param_distr,
    n_jobs=1,
    cv=3,
    verbose=1,
    scoring='neg_mean_absolute_error')  # neg_mean_squared_error

st = time.time()
grid_search.fit(X_train, y_train)
logging.info(f'GridSearchCV runtime: {timedelta(seconds=time.time() - st)} per {n_iter_search} setting dei parametri')
logging.info(f'GridSearchCV best score: {grid_search.best_score_}')
logging.info(f'GridSearchCV best parameters: {grid_search.best_params_}')
logging.info(f'GridSearchCV results: {grid_search.cv_results_}')

In [ ]:
print(f'GridSearchCV runtime: {timedelta(seconds=time.time() - st)} per {n_iter_search} setting dei parametri')
print(f'GridSearchCV best estimator: {grid_search.best_estimator_}')   # this method returns a trained model
print(f'GridSearchCV best parameters: {grid_search.best_params_}')
print(f'GridSearchCV best score: {grid_search.best_score_}')
print(f'GridSearchCV results: {grid_search.cv_results_}')

### Halving Grid Search Cross Validation

This method is similar to Grid Search Cross Validation but it follows a successive halving approach in the following steps:
- it trains a subset of data to all the combinations of parameters;
- top performing candidates or combinations are selected;
- a larger subset of training data is trained on the top-performing candidates
- the above 3 steps are repeated until the best set of hyperparameters are left standing.

Using Halving Grid Search, for each passing iteration, the parameter components are decreasing and the training dataset is increasing.
Since the algorithm follows a successive halving approach, so the time complexity of the algorithm is comparatively very less compared to Grid Search Cross Validation.

In [ ]:
param_grid = {
    'max_depth': [3, 4, 7, 10, 25],
    'gamma': [0.5, 1, 5, 10, 25],
    'min_child_weight': [1, 3, 5, 10, 25],
    'reg_lambda': [5, 10, 50, 100, 300],
    'scale_pos_weight': [1, 3, 5, 10, 25]
}

halving_search = HalvingGridSearchCV(
    xgb.XGBClassifier(objective="binary:logistic"), param_grid, scoring="roc_auc", n_jobs=-1, min_resources="exhaust", factor=3)
halving_search.fit(X_train, y_train)
logging.info(f'HalvingGridSearchCV runtime: {timedelta(seconds=time.time() - st)} per {n_iter_search} setting dei parametri')
logging.info('HalvingGridSearchCV best parameters: ', halving_cv.best_params_)
logging.info('HalvingGridSearchCV best score: ', halving_cv.best_score_)

### Random Search Cross Validation

This method is useful when dealing with large hyperparameter search space since it evalueates a given number of random combinations by selecting a random value for each hyperparameter at every iteration.  

Benefits:
- by simply setting the number of iterations, you have more control over the computing budget you want to allocate to hyperparameter search

In [ ]:
n_iter_search = 10
param_distr = {
    'n_estimators': [400, 600, 800, 1000, 1200],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.001, 0.005, 0.01],
    'gamma': [0.1, 0.5, 1.0]
}

rnd_search = RandomizedSearchCV(
    XGBRegressor(verbosity=0, random_state=123),
    param_distributions=param_distr,
    n_iter=n_iter_search,
    n_jobs=2,
    cv=3,
    verbose=1,
    scoring='neg_mean_squared_error')

st = time.time()
rnd_search.fit(X_train, y_train)
logging.info(f'RandomizedSearchCV runtime: {timedelta(seconds=time.time() - st)} per {n_iter_search} setting dei parametri')
logging.info('RandomizedSearchCV best score: ', rnd_search.best_score_)
logging.info('RandomizedSearchCV best parameters: ', rnd_search.best_params_)

<a name="optuna"></a>
## Optuna

[Return to Contents](#contents)

Optuna is an open source hyperparameter optimization framework.  
It uses a history record of trials to determine which hyperparameter values to try next.
Using this data, it estimates a promising area and tries values in that area.  
Optuna then estimates an even more promising region based on the new result.
It repeats this process using the history data of trials completed thus far.
Specifically, it employs a **Bayesian optimization algorithm** called **Tree-structured Parzen Estimator**.

To use Optuna you first need to define the objective function for Optuna to maximize.
The objective function should take a `Trial` object as the input and return the score, a float value or a list of float values.

The next step is to use the objective function to create a `Study` object and then optimize it.

**Note** that
- you can optimize whatever metric you like  
- you can also run multi-objective optimization jobs simply by specifying multiple output metrics to the objective function and then you have to specify the optimization direction for each of them

**Libraries**
- https://optuna.readthedocs.io/en/stable/

**References**
- https://www.kaggle.com/code/hamzaghanmi/xgboost-catboost-using-optuna?scriptVersionId=94510532
- https://optuna.readthedocs.io/en/latest/faq.html#how-do-i-suggest-variables-which-represent-the-proportion-that-is-are-in-accordance-with-dirichlet-distribution

In [ ]:
def objective(trial: optuna.Trial, train_features, train_labels):
    # use the trial obj to suggest a range and a type for each hyperparameter
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 5, 15)
    n_estimators = trial.suggest_int("n_estimators", 10, 250)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 5, 20)
    # # you can also suggest float parameters with one of the following commands
    # uniform_param = trials.suggest_uniform()
    # loguiniform_param = trials.suggest_loguniform()
    # discrete_uniform_param = trials.suggest_discrete_uniform()
    
    # define the model with the suggested ranges and types for each hyperparameter to be optimized by optuna
    model = RandomForestClassifier(
        criterion=criterion,
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth,
        n_jobs=n_jobs,
    )
    
    # split data in order to train and test the model
    train_x, valid_x, train_y, valid_y = train_test_split(
        train_features,
        train_labels,
        test_size=0.25,
        random_state=123,
        stratify=train_labels,
    )
    model.fit(train_x, train_y)
    y_pred = model.predict(valid_x)
    
    # otherwise you can consider using cross validation as follows
    # scores = cross_val_score(model, x, y, cv=KFold(n_splits=10, shuffle=True, random_state=123), scoring='neg_root_mean_squared_error')
    # return scores.mean()
    
    # add user defined attributes to the study
    # these attributes will appear with the "user_attrs_" prefix in the study
    trial.set_user_attr("class_report_validation", classification_report(valid_y, y_pred, output_dict=True))
    trial.set_user_attr("class_report_training", classification_report(train_y, model.predict(train_x), output_dict=True))
    trial.set_user_attr("model", model)
    trial.set_user_attr("name", 'RandForest')
    trial.set_user_attr("id", trial.number)
    
    # the output of the objective function has to me a number that optuna will optimize
    # you can pick whatever metric, score or value
    return f1_score(valid_y, y_pred, average="weighted")

In [ ]:
obj = partial(objective, train_features=X, train_labels=y)

study = optuna.create_study(direction='maximize')
study.optimize(obj, n_trials=20, n_jobs=4)
study_df = study.trials_dataframe().sort_values(by='value', ascending=False).reset_index(drop=True)

`plot_optimization_history` shows the scores from all trials as well as the best score so far at each point.

In [ ]:
optuna.visualization.plot_optimization_history(study)

`plot_parallel_coordinate` interactively visualizes the hyperparameters and scores

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

`plot_slice` shows the evolution of the search.
You can see where in the hyperparameter space your search went and which parts of the space were explored more

In [ ]:
optuna.visualization.plot_slice(study)

## Hyperopt

## Other methods

These inclue sklearn `RidgeCV`, `LassoCV`, and `ElasticNetCV` and Ray `Tune` (https://docs.ray.io/en/latest/tune/index.html)

---
<a name="perf"></a>
# Performances

[Return to Contents](#contents)

**Remember** to always inspect feature importance as you might have forgot a relevant feature that had to be discarded as the target variable or some feature derived from it.

**References**
- https://towardsdatascience.com/avoid-r-squared-to-judge-regression-model-performance-5c2bc53c8e2e

### Regression

In [ ]:
score = rmse_cv(model, X_train.values, y_train)
print(f'Model score: {score.mean():.4f} ({score.std():.4f})')

In [ ]:
show_feature_importances(xgb_model, X_train.columns.tolist())

In [ ]:
print('TRAIN PERFORMANCES')
regression_performances(y_train, y_train_pred, metrics=['mae', 'rmse'])
print('TEST PERFORMANCES')
regression_performances(y_val, y_val_pred, metrics=['mae', 'rmse'])

In [ ]:
plot_regression_errors(xgb_model, X_train, X_val, y_train, y_val, target_='fatturato')

### Classification

In [ ]:
print('CLASSIFICATION PERFORMANCES')
classification_performances(y, y_pred)
                            
# to get performances in pandas DataFrame
perf = pd.Series(classification_performances(y, y_pred, verbose=0)).to_frame().T
display(perf)

In [ ]:
# to plot performances
plot_classification_performances(y, y_pred)

<a name="shap"></a>
## SHAP

[Return to Contents](#contents)

SHAP stands for **SHapley Additive exPlanations**.

SHAP values are based on **Shapley values**, a concept coming from game theory that quantifies the contribution that each player brings to the result of a game.
Similarly, SHAP values quantify the contribution that each feature brings to the prediction made by a machine learning model.  
Note that here the game is a **single observation** so keep in mind that SHAP values always depend on the machine learning model and on the observation. Indeed, SHAP is about **local interpretability** of a predictive model.  

In game theory, Shapley values are calculated by averaging the contribution of every subset of players to the game. This is matematically represented by the expected value conditioned by the given subset of features. But note that, in presence of interaction, the conditioned expected values depends on the order of the conditions meaning that in order to get the exact result one should compute all the *n!* permutations of features.  
In order to simplify the calculus, SHAP values assume that the features are **independent** of each other.

**Libraries**
- https://shap.readthedocs.io/en/latest/index.html

**References**
- https://towardsdatascience.com/interpretable-machine-learning-using-shap-theory-and-applications-26c12f7a7f1a
- https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30#:~:text=SHAP%20%E2%80%94%20which%20stands%20for%20SHapley,output%20of%20any%20predictive%20algorithm.

**Watch out**: SHAP strongly depends on some assumptions therefore you should understand them deeply in order to use SHAP. In particular
- correlation not causality: SHAP only explains the variable correlation defined per the model structure
- dependency on the model: SHAP by design represents how important it is a feature to the model, not how important it actually is
- multicollinearity issue: when having variables with high degree of multicollinearity the SHAP values are high for one and very low for the remaining ones

**References**
- https://towardsdatascience.com/why-shap-values-might-not-be-perfect-cbc8056056be
- https://towardsdatascience.com/using-shap-for-explainability-understand-these-limitations-first-1bed91c9d21#:~:text=Limitation%201%3A%20Correlation%20not%20Causality,would%20have%20causality%20as%20well
- https://towardsdatascience.com/using-shap-values-to-explain-how-your-machine-learning-model-works-732b3f40e137

First load JS visualization code to notebook

In [ ]:
shap.initjs()

**explainer**

In [ ]:
explainer = shap.Explainer(model=cbc_model, model_output='raw')
explainer = shap.TreeExplainer(model=model, model_output='raw')

**expected values**

expected value is $E[f(x)]$ that is the expected value of the target variable, or in other words, the mean of all predictions

In [ ]:
print(explainer.expected_value)
print(expit(explainer.expected_value))

**shap values**

In [ ]:
shap_values = explainer.shap_values(X_train)

As we said, SHAP values represent the impact of a feature for a specific observation, hence SHAP values differ depending on the chosen observation.  
A plot that allows to get an insight of SHAP values over the full set of observatoins is **beeswarm plot**.

Beeswarm plot represents the SHAP value of a feature for each observation as a point.

The beeswarm plot's straight vertical line marks the null value.
Features names are printed next to the plot for reference.
For each feature, the shap values for each observation for the specific feature are represented as dots or as violin in an horizontal line where values that are on the right side have a high SHAP value whereas values that are on the left side have a low SHAP value.
Each dot is colored depending on the observation value for the specific feature, blue denoting a low value for that feature and red denoting a high value insted.


In [ ]:
shap.summary_plot(
    shap_values=shap_values,
    features=train_X,
    max_display=100,
    plot_type='dot',
    plot_size=(30,25))

Another useful representation is the **force plot**.

Force plots depicts the contribution of each feature to the model's prediction for a specific observation.  
This allows to perfectly explain how your model arrived at the prediction it did.

The force plot represents both the model's base value as well as the prediction for the given observation.
The most significant features that are driving the prediction are printed below.
Features that appear in red color on the left side are the ones pushing the predicted value up whereas the features that appear in blue color on the right side are the ones pulling the predicted value down.
On each side, the most significant features are represented.

In [ ]:
supp = train_X[train_X.index == row.ADMIN6_CODE]
proba = model.predict_proba(supp.iloc[0,:])[1]

shap.plots.force(explainer(supp))

# for a finer control over the parameters
shap_values = explainer.shap_values(supp)
shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,                     # shap_values[row['index']],
    features=supp.iloc[0,:],                     # train_X.iloc[row['index']],
    feature_names=supp.columns,                  # train_X.columns,
    text_rotation=0,
    link='logit'                                 # identity or logit, with identity the result f(x) might be outside (0, 1)
)

You can save the plot as html file by simply assigning the plot to a variable and then using the method `save_html`

Finally, the latest added plot to the SHAP library is **decision plot**.

Decision plot offer a detailed view of a model prediction for a specific observation.

The decision plot's straight vertical line marks the model's base value.
The colored line instead is the prediction for each feature and feature values are printed next to the prediction line for reference.  
Starting at the bottom of the plot, the prediction line shows how the SHAP values accumulate from the base value to arrive at the model's prediction

In [ ]:
print(f'SHAP decision plot for PDV {high_code} (idx {high_idx})')
shap.decision_plot(
    base_value=explainer.expected_value,
    shap_values=explainer.shap_values(train_X[train_X.index == high_code]),
    feature_names=list(train_X.columns),
    link='logit'
)

Other plots

In [ ]:
# bar plot
shap.plots.bar(explainer(supp)[0], max_display=15)

# waterfall plot
shap.plots.waterfall(explainer(supp[0, cols])

<a name="lift"></a>
## Lift curve

[Return to Contents](#contents)

---
<a name="tf"></a>
# TensorFlow

[Return to Contents](#contents)

In [ ]:
print("TensorFlow version:", tf.__version__)

TensorFlow 2 by default has eager execution enabled that makes TensorFlow functions execute operations immediately as opposed to adding to a graph to be executed later in a tf.compat.v1.Session and return concrete values as opposed to symbolic references to a node in a computational graph

In [ ]:
tf.compat.v1.disable_eager_execution()

Define TensorFlow edges: variables

In [ ]:
# constants are immutable values which do not change during a computation process
A = tf.constant(5.5, name='constant_a')
B = tf.constant(3., name='constant_b')
C = tf.constant(0.7, name='constant_c')

# placeholders are values assigned once and that do not change afterwars
# placeholders in tensorflow do not have input values even though you can specify some if you want
# they serve as input nodes, they are actually tensors which do not take their values until we execute the graph
x = tf.compat.v1.placeholder(tf.float32, shape=[3], name='x')
y = tf.compat.v1.placeholder(tf.float32, shape=[3], name='y')

# variables are values that can be constantly changed or recomputed
m = tf.Variable([2.5, 4.0, 2.0], tf.float32, name='var_m')
c = tf.Variable([5.0, 10.0, 3.5], tf.float32, name='var_c')
number = tf.Variable(4.5)
multiplier = tf.Variable(1.7)

If you use variables in your TensorFlow program you need to run a special command to initialize variables.  
Variables cannot be used before running this specific command

In [ ]:
# to initialize all variables in your code
init = tf.compat.v1.global_variables_initializer()

# to only initialize a bunch of selected variables
tf.compat.v1.variables_initializer([m, c])

Define TensorFlow nodes: operations

In [ ]:
add = tf.add(A, B, name='add_ab')
subtract = tf.subtract(B, C, name='subtract_bc')
square = tf.square(C, name='square_c')
final_sum = tf.add_n([add, subtract, square], name='final_sum')

sum_x = tf.reduce_sum(x, name='sum_x')
prod_y = tf.reduce_prod(y, name='prod_y')
final_div = tf.math.divide(sum_x, prod_y, name='final_div')

# an operation on variables and placeholders will also be a variable
# NOTE: to compute a combined operation of variables and placeholders, they must have the same shape
line = m * x + c

result = number.assign(tf.multiply(number, multiplier))

Create TensorFlow session

In [ ]:
# NOTE: remember to instantiate the TensorFlow session using with operator in order to be sure it is automatically shut down when you are done
with tf.compat.v1.Session() as sess:
    # run init to initialize all variables
    sess.run(init)
    
    print(f'sum(x): {sess.run(sum_x, feed_dict={x: [100,200,300]})}')
    print(f'prod(y): {sess.run(prod_y, feed_dict={y: [11,22,33]})}')
    print(f'sum(x)/prod(y): {sess.run(final_div, feed_dict={x: [100,200,300], y: [11,22,33]})}', end='\n\n')
    
    print(f'Line mx + c: {sess.run(line, feed_dict={x: [0, 1, 10]})}', end='\n\n')
    
    for i in range(5):
        print(f'Result number*multiplier: {sess.run(result)}')
        print(f'Increase multiplier value: {sess.run(multiplier.assign_add(1))}')

To visualize computation graph using TensorBoard you need to use tf.summary.FileWriter to save data in a target folder and then initialize TensorBoard on that folder

In [ ]:
writer = tf.summary.FileWriter('./TensorFlowOutput', sess.graph)
writer.close()

# WATCH OUT: this is google.datalab.ml code, however Datalab has been shutdown by google
tensorboard_pid = ml.TensorBoard.start('./TensorFlowOutput')

Once you're done close the TensorBoard session

In [ ]:
ml.TensorBoard.stop(tensorboard_pid)

<a name="tf_mnist"></a>
## MNIST

[Return to Contents](#contents)

In [ ]:
# load and prepare mnist data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
# build a feed forward neural network
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
# to run the model on a single data point
predictions = model(x_train[:1]).numpy()
tf.nn.softmax(predictions).numpy()

In [ ]:
# define a loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# configure and compile the model specifying an optimizer, the loss function and an evaluation metric
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

In [ ]:
# train the model
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
# in order to retrieve probabilities you can wrap the model with the softmax function
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
probability_model(x_test[:5])